In [217]:
import pandas as pd
import numpy as np
pd.set_option('display.max_columns', None)  # Not enough people know this exists
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline
from scipy.stats import pearsonr
from scipy.stats import norm
# Import ML
from xgboost import XGBRegressor
from sklearn import preprocessing, model_selection
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, mean_squared_log_error
from sklearn.svm import LinearSVR,SVR
from sklearn.ensemble import RandomForestRegressor
from sklearn.decomposition import PCA
from sklearn.manifold import TSNE
from sklearn.feature_selection import SelectKBest, chi2
LabelEnc = preprocessing.LabelEncoder()
OHE = preprocessing.OneHotEncoder()

In [147]:
train_data = pd.read_csv(r'E:/Downloads/Housing/train.csv')
test_data = pd.read_csv(r'E:/Downloads/Housing/test.csv')

In [52]:
def outlier_and_nans(train):
    """ Remove outliers and nans from training data"""
    train = train.drop(index=train.GrLivArea.nlargest(2).index)
    train = train.drop(index=train.SalePrice.nlargest(1).index)
    train = train.dropna(subset=['MasVnrArea'])  # Around 8 examples
    return train

In [161]:
def fill_nans(test):
    test['MasVnrArea'].fillna(test['MasVnrArea'].mean(), inplace=True)
    test['GarageArea'].fillna(test['GarageArea'].mean(), inplace=True)
    test['TotalBsmtSF'].fillna(test['TotalBsmtSF'].mean(), inplace=True)

In [188]:
def feature_extraction(data, dummies):
    X = pd.DataFrame(index=data.index)
    X['OverallQual'] = preprocessing.scale(data['OverallQual'])  # Maybe scale it later
    X['GrLivArea'] = preprocessing.scale(np.log(data['GrLivArea']))
    X['GarageArea'] = preprocessing.scale(data['GarageArea'].apply(lambda x: x if x == 0 else np.log(x)))
    X['TotalBsmtSF'] = preprocessing.scale(data['TotalBsmtSF'].apply(lambda x: x if x == 0 else np.log(x)))
    X['FullBaths'] = preprocessing.scale(data['FullBath'])  # Maybe scale it later
    X['AgeWhenSold'] = preprocessing.scale(data['YrSold'] - data['YearBuilt'])
    X['MasVnrArea'] = preprocessing.scale(data['MasVnrArea'].apply(lambda x: x if x == 0 else np.log(x)))
    data['HasFirePlace'] = 0
    data.loc[data['Fireplaces'] != 0,'HasFirePlace'] = 1
    X['HasFirePlace'] = data['HasFirePlace']
    X = pd.concat([X,pd.get_dummies(data[dummies])], axis=1)
    return X

In [249]:
def fit_predict(train, test, model):
    X_train, y_train, X_val, y_val, X_test = prepare_for_model(train, test, train["SalePrice"])
    model.fit(X_train, y_train)
    print("Train error: ",np.sqrt(mean_squared_log_error(np.exp(y_train),np.exp(model.predict(X_train)))))
    print("Val error: ", np.sqrt(mean_squared_log_error(np.exp(y_val), np.exp(model.predict(X_val)))))
    pred = np.exp(pd.Series(model.predict(X_test)))
    pred.name = "SalePrice"
    return pd.concat([test.Id,pred],axis = 1)

In [250]:
def prepare_for_model(train, test, tag):
    dummies = train.dtypes[train.dtypes == object].index
    X_train = feature_extraction(train, dummies)
    y_train = np.log(tag)
    X_test = feature_extraction(test, dummies)
    X_train = X_train[X_test.columns]
    X_test = X_test[X_train.columns]
    X_train, X_val, y_train, y_val = model_selection.train_test_split(X_train, y_train, test_size= 0.15)
    return X_train, y_train, X_val, y_val, X_test
    
    

In [96]:
print(train_data.shape)
train_r = outlier_and_nans(train_data)
print(train_r.shape)

(1460, 81)
(1449, 81)


In [162]:
fill_nans(test_data)

In [251]:
model = SVR(C=0.75)
fit_predict(train_r ,test_data ,model).to_csv(r'out.csv',index=False)

KeyError: "['GarageQual_Ex', 'Exterior1st_ImStucc', 'Heating_Floor', 'Exterior2nd_Other', 'Condition2_RRAn', 'HouseStyle_2.5Fin', 'Condition2_RRNn', 'Electrical_Mix', 'RoofMatl_Membran'] not in index"

In [237]:
f = pd.get_dummies(train_r[train_r.dtypes[train_r.dtypes == object].index])
f

,MSZoning_C (all),MSZoning_FV,MSZoning_RH,MSZoning_RL,MSZoning_RM,Street_Grvl,Street_Pave,Alley_Grvl,Alley_Pave,LotShape_IR1,LotShape_IR2,LotShape_IR3,LotShape_Reg,LandContour_Bnk,LandContour_HLS,LandContour_Low,LandContour_Lvl,Utilities_AllPub,Utilities_NoSeWa,LotConfig_Corner,LotConfig_CulDSac,LotConfig_FR2,LotConfig_FR3,LotConfig_Inside,LandSlope_Gtl,LandSlope_Mod,LandSlope_Sev,Neighborhood_Blmngtn,Neighborhood_Blueste,Neighborhood_BrDale,Neighborhood_BrkSide,Neighborhood_ClearCr,Neighborhood_CollgCr,Neighborhood_Crawfor,Neighborhood_Edwards,Neighborhood_Gilbert,Neighborhood_IDOTRR,Neighborhood_MeadowV,Neighborhood_Mitchel,Neighborhood_NAmes,Neighborhood_NPkVill,Neighborhood_NWAmes,Neighborhood_NoRidge,Neighborhood_NridgHt,Neighborhood_OldTown,Neighborhood_SWISU,Neighborhood_Sawyer,Neighborhood_SawyerW,Neighborhood_Somerst,Neighborhood_StoneBr,Neighborhood_Timber,Neighborhood_Veenker,Condition1_Artery,Condition1_Feedr,Condition1_Norm,Condition1_PosA,Condition1_PosN,Condition1_RRAe,Condition1_RRAn,Condition1_RRNe,Condition1_RRNn,Condition2_Artery,Condition2_Feedr,Condition2_Norm,Condition2_PosA,Condition2_PosN,Condition2_RRAe,Condition2_RRAn,Condition2_RRNn,BldgType_1Fam,BldgType_2fmCon,BldgType_Duplex,BldgType_Twnhs,BldgType_TwnhsE,HouseStyle_1.5Fin,HouseStyle_1.5Unf,HouseStyle_1Story,HouseStyle_2.5Fin,HouseStyle_2.5Unf,HouseStyle_2Story,HouseStyle_SFoyer,HouseStyle_SLvl,RoofStyle_Flat,RoofStyle_Gable,RoofStyle_Gambrel,RoofStyle_Hip,RoofStyle_Mansard,RoofStyle_Shed,RoofMatl_CompShg,RoofMatl_Membran,RoofMatl_Metal,RoofMatl_Roll,RoofMatl_Tar&Grv,RoofMatl_WdShake,RoofMatl_WdShngl,Exterior1st_AsbShng,Exterior1st_AsphShn,Exterior1st_BrkComm,Exterior1st_BrkFace,Exterior1st_CBlock,Exterior1st_CemntBd,Exterior1st_HdBoard,Exterior1st_ImStucc,Exterior1st_MetalSd,Exterior1st_Plywood,Exterior1st_Stone,Exterior1st_Stucco,Exterior1st_VinylSd,Exterior1st_Wd Sdng,Exterior1st_WdShing,Exterior2nd_AsbShng,Exterior2nd_AsphShn,Exterior2nd_Brk Cmn,Exterior2nd_BrkFace,Exterior2nd_CBlock,Exterior2nd_CmentBd,Exterior2nd_HdBoard,Exterior2nd_ImStucc,Exterior2nd_MetalSd,Exterior2nd_Other,Exterior2nd_Plywood,Exterior2nd_Stone,Exterior2nd_Stucco,Exterior2nd_VinylSd,Exterior2nd_Wd Sdng,Exterior2nd_Wd Shng,MasVnrType_BrkCmn,MasVnrType_BrkFace,MasVnrType_None,MasVnrType_Stone,ExterQual_Ex,ExterQual_Fa,ExterQual_Gd,ExterQual_TA,ExterCond_Ex,ExterCond_Fa,ExterCond_Gd,ExterCond_Po,ExterCond_TA,Foundation_BrkTil,Foundation_CBlock,Foundation_PConc,Foundation_Slab,Foundation_Stone,Foundation_Wood,BsmtQual_Ex,BsmtQual_Fa,BsmtQual_Gd,BsmtQual_TA,BsmtCond_Fa,BsmtCond_Gd,BsmtCond_Po,BsmtCond_TA,BsmtExposure_Av,BsmtExposure_Gd,BsmtExposure_Mn,BsmtExposure_No,BsmtFinType1_ALQ,BsmtFinType1_BLQ,BsmtFinType1_GLQ,BsmtFinType1_LwQ,BsmtFinType1_Rec,BsmtFinType1_Unf,BsmtFinType2_ALQ,BsmtFinType2_BLQ,BsmtFinType2_GLQ,BsmtFinType2_LwQ,BsmtFinType2_Rec,BsmtFinType2_Unf,Heating_Floor,Heating_GasA,Heating_GasW,Heating_Grav,Heating_OthW,Heating_Wall,HeatingQC_Ex,HeatingQC_Fa,HeatingQC_Gd,HeatingQC_Po,HeatingQC_TA,CentralAir_N,CentralAir_Y,Electrical_FuseA,Electrical_FuseF,Electrical_FuseP,Electrical_Mix,Electrical_SBrkr,KitchenQual_Ex,KitchenQual_Fa,KitchenQual_Gd,KitchenQual_TA,Functional_Maj1,Functional_Maj2,Functional_Min1,Functional_Min2,Functional_Mod,Functional_Sev,Functional_Typ,FireplaceQu_Ex,FireplaceQu_Fa,FireplaceQu_Gd,FireplaceQu_Po,FireplaceQu_TA,GarageType_2Types,GarageType_Attchd,GarageType_Basment,GarageType_BuiltIn,GarageType_CarPort,GarageType_Detchd,GarageFinish_Fin,GarageFinish_RFn,GarageFinish_Unf,GarageQual_Ex,GarageQual_Fa,GarageQual_Gd,GarageQual_Po,GarageQual_TA,GarageCond_Ex,GarageCond_Fa,GarageCond_Gd,GarageCond_Po,GarageCond_TA,PavedDrive_N,PavedDrive_P,PavedDrive_Y,PoolQC_Ex,PoolQC_Fa,PoolQC_Gd,Fence_GdPrv,Fence_GdWo,Fence_MnPrv,Fence_MnWw,MiscFeature_Gar2,MiscFeature_Othr,MiscFeature_Shed,MiscFeature_TenC,SaleType_COD,SaleType_CWD,SaleType_Con,SaleType_ConLD,SaleType_ConLI,SaleType_ConLw,SaleType_New,SaleType_Oth,SaleType_WD,SaleCondition_Abnorml,Sale

In [247]:
k = SelectKBest(chi2, k=100).fit(f, train_r['SalePrice'])
r = f.iloc[:,k.get_support(indices=True)].columns

In [235]:
train_r.isnull().sum()

Id                  0
MSSubClass          0
MSZoning            0
LotFrontage       257
LotArea             0
Street              0
Alley            1359
LotShape            0
LandContour         0
Utilities           0
LotConfig           0
LandSlope           0
Neighborhood        0
Condition1          0
Condition2          0
BldgType            0
HouseStyle          0
OverallQual         0
OverallCond         0
YearBuilt           0
YearRemodAdd        0
RoofStyle           0
RoofMatl            0
Exterior1st         0
Exterior2nd         0
MasVnrType          0
MasVnrArea          0
ExterQual           0
ExterCond           0
Foundation          0
                 ... 
KitchenAbvGr        0
KitchenQual         0
TotRmsAbvGrd        0
Functional          0
Fireplaces          0
FireplaceQu       686
GarageType         81
GarageYrBlt        81
GarageFinish       81
GarageCars          0
GarageArea          0
GarageQual         81
GarageCond         81
PavedDrive          0
WoodDeckSF